In [2]:
%cd ..

/Users/juansegundohevia/Documents/repos/ConceptGuidedColorization


In [4]:
from palette_model import ColorizationModel as model
import os 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

In [5]:
CKP_PATH = "weights/ColorizationModel/"

In [8]:
G = model.UNet(imsize=256, add_L=True)
D = model.Discriminator(add_L=True , imsize=256)

# load weights
G.load_state_dict(torch.load(os.path.join(CKP_PATH, "8_G.ckpt"), map_location=torch.device('cpu')))
D.load_state_dict(torch.load(os.path.join(CKP_PATH, "8_D.ckpt"), map_location=torch.device('cpu')))

/var/folders/2c/s1dc7gqn0jz4dls2kprq822m0000gn/T/ipykernel_47945/917714610.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  G.load_state_dict(torch.load(os.path.join(CKP_

<All keys matched successfully>